In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

## Após importar as bibliotecas, vamos ler a base de dados e mudar o atributo 'sex' para a ultima posição, mudando a tarefa de regressão para classificação

In [6]:
df = pd.read_csv("regression_normalized.csv")
df = df.reindex(columns=['age', 'bmi', 'children', 'smoker', 'region', 'charges', 'sex'])

In [7]:
df.head()

,age,bmi,children,smoker,region,charges,sex
0,0.021739,0.321227,0.0,1.0,1.000000,0.251611,0.0
1,0.000000,0.479150,0.2,0.0,0.666667,0.009636,1.0
2,0.217391,0.458434,0.6,0.0,0.666667,0.053115,1.0
3,0.326087,0.181464,0.0,0.0,0.333333,0.333010,1.0
4,0.304348,0.347592,0.0,0.0,0.333333,0.043816,1.0


In [8]:
X_, y_ = df.iloc[:,:-1], df.iloc[:, -1]

In [9]:
print(df.shape)

(1338, 7)


## Agora vamos testar um classificador para estipular um baseline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3, random_state=1)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Resultado sem redução de dimensionalidade:')
print(classification_report(y_test, y_pred))

Resultado sem redução de dimensionalidade:
              precision    recall  f1-score   support

         0.0       0.63      0.59      0.61       205
         1.0       0.60      0.64      0.62       197

    accuracy                           0.61       402
   macro avg       0.61      0.61      0.61       402
weighted avg       0.61      0.61      0.61       402



## Primeiramente vamos reduzir a dimensão por variância

In [11]:
selector = VarianceThreshold()
new_x = selector.fit_transform(X_)
print(new_x.shape)

(1338, 6)


### Nota-se que a redução não surtiu efeito, pois neste caso, estamos lidando somente com features contínuas


## Agora vamos tentar reduzir a dimensionalidade por uma abordagem estatística. Primeiro para 2 e depois para 3 dimensões, variando o valor de K. Lembre-se que após gerar um novo conjunto X reduzido, você deverá testar o mesmo classificador utilizado no exemplo inicial para fins de comparação:

In [13]:
x_new = SelectKBest(chi2, k=2).fit_transform(X_, y_)
print(x_new.shape)

(1338, 2)


array([[1.        , 0.25161076],
       [0.        , 0.00963595],
       [0.        , 0.05311516],
       ...,
       [0.        , 0.00810808],
       [0.        , 0.01414352],
       [1.        , 0.44724873]])

In [19]:
def reduceWithScore():
    for k in range(2, 7):
        x_new = SelectKBest(chi2, k=k).fit_transform(X_, y_)
        X_train, X_test, y_train, y_test = train_test_split(x_new, y_, test_size=0.3, random_state=1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print('Resultado com redução de dimensionalidade= ', k)
        print(classification_report(y_test, y_pred))

## Agora você deverá fazer o mesmo para 3 dimensões

In [21]:
reduceWithScore ()

Resultado com redução de dimensionalidade=  2
              precision    recall  f1-score   support

         0.0       0.68      0.62      0.65       205
         1.0       0.64      0.69      0.66       197

    accuracy                           0.66       402
   macro avg       0.66      0.66      0.66       402
weighted avg       0.66      0.66      0.66       402

Resultado com redução de dimensionalidade=  3
              precision    recall  f1-score   support

         0.0       0.57      0.54      0.55       205
         1.0       0.55      0.58      0.56       197

    accuracy                           0.56       402
   macro avg       0.56      0.56      0.56       402
weighted avg       0.56      0.56      0.56       402

Resultado com redução de dimensionalidade=  4
              precision    recall  f1-score   support

         0.0       0.62      0.60      0.61       205
         1.0       0.60      0.61      0.60       197

    accuracy                           0.61 

## Perguntas a serem respondidas:
### Existe melhora na performance do classificador ao reduzir a dimensionalidade? Para o método SelectKBest, qual o valor de K que maximiza a performance? 



- Foi evidente que a redução de dimensionalidade com k  2 foi a melhor tanto em acuracia como nas outras métricas
- Realmente o resultado me impressionou, o dataset foi escolhido seletivamente para ter esse resultado